# Juliaで固有値問題を精度保証付き数値計算したい


## 固有値問題とは

固有値問題は与えられた行列の固有値および固有ベクトルを求めること。つまり、
$
\newcommand{\C}{\mathbb{C}}
\newcommand{\IC}{\mathbb{IC}}
\newcommand{\bX}{\bar{X}}
$

$$
Ax=\lambda x,\quad A\in \C^{n\times n}
$$

を満たす$\lambda\in \C$, $x\in \C^{n\times n}$ ($x\neq 0$) を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、`LinearAlgebra`パッケージに含まれている関数`eigen`・`eigvals`・`eigvecs`で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |

多分、Lapackの何かを呼び足してるはず。

In [61]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [62]:
val#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [63]:
vec#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [64]:
eigvals([1 2; 2 1])#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [65]:
eigvecs([1 2; 2 1])#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

## 密行列の固有値問題に対する精度保証

密行列とは、行列内のほとんどの要素が0でない行列のことを指す。ここでは、行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ の全ての固有値に対する精度保証アルゴリズムを考える。 

### 実装したいアルゴリズム
(1)
ある行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

(2)
$A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

(3)
区間連立1次方程式 $\bX G=C$ の解集合の包含 $G\supset\bX^{-1}\cdot C$ を求める。

(4)
$G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。

#### (1)行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ が対角化可能であるとするとき、$\lambda_i$ に対する固有ベクトル$x^{(i)}\in\C^{n}$ $(i=1,2,\cdots,n)$ を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\C^{n\times n}
$$

によって、$A$ は

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [66]:
using LinearAlgebra

n =100;
A = randn(n,n);

λ, X = eigen(A);

In [67]:
λ #固有値

100-element Array{Complex{Float64},1}:
 -10.766022723425223 - 0.8122874492930389im
 -10.766022723425223 + 0.8122874492930389im
  -8.901198365927415 - 2.472701572517997im
  -8.901198365927415 + 2.472701572517997im
  -7.796420635615081 - 6.605672668300338im
  -7.796420635615081 + 6.605672668300338im
   -7.39292803449457 - 1.0096385968492632im
   -7.39292803449457 + 1.0096385968492632im
  -7.326769092147958 + 0.0im
  -7.154711828575033 - 5.267615306775259im
  -7.154711828575033 + 5.267615306775259im
  -6.628212798009513 - 3.3917662709575174im
  -6.628212798009513 + 3.3917662709575174im
                     ⋮
   6.643063450796827 - 0.6679521100997574im
   6.643063450796827 + 0.6679521100997574im
   7.138270608605898 - 6.496864665555203im
   7.138270608605898 + 6.496864665555203im
   7.565808571113444 - 3.5776076639182754im
   7.565808571113444 + 3.5776076639182754im
   7.925657360977363 - 0.5526696884902491im
   7.925657360977363 + 0.5526696884902491im
   8.305101345408058 - 2.833982915383

In [68]:
X #固有ベクトル

100×100 Array{Complex{Float64},2}:
 -0.0137179-0.008181im    …   0.00367427-0.0216093im
  0.0127764+0.0208956im       -0.0358408+0.0897729im
 -0.0320241-0.0347265im       -0.0284699-0.0358665im
 -0.0849695+0.00282483im       0.0831393+0.0348483im
    -0.2418-0.0im              0.0116357-0.071323im
  -0.160058-0.0326811im   …   -0.0297767+0.0398088im
 -0.0537056+0.0537747im         0.010946+0.0273993im
  0.0312759-0.00732992im        0.034381+0.02923im
   0.054274+0.0696518im         0.110116+0.0248822im
 -0.0243595+0.00054807im      0.00141307-0.0241246im
  0.0515867+0.00203391im  …   -0.0225958+0.0810808im
  0.0875125-0.0735259im       -0.0312765-0.00474837im
   0.114008-0.00602452im       0.0520928-0.020064im
           ⋮              ⋱  
 -0.0449836-0.0277217im        0.0243167-0.00615177im
   -0.12139-0.00179806im        0.167622+0.0406149im
    0.11091+0.00752602im  …    0.0557256-0.0379002im
 -0.0884635+0.075204im        -0.0655205+0.0454215im
 -0.0220236+0.0278526im        0.013

しかし、実際のところ上記の数値計算で得られる固有ベクトル $X$ は近似の値 $\bX\in \C^{n\times n}$ (以下、近似固有ベクトルという) である。しかし、近似固有ベクトルを並べた行列を使って、対角化したものについては、対角優位性が期待できる。この行列を $G$ とすると、

$$
G := \bX^{-1}A\bX
$$

となり、行列 $G$ の対角成分は行列 $A$ の固有値の近似となる。

また、対角優位性とは、ある行列$A=(a_{ij})$について、

$$
\left|a_{i i}\right|>\sum_{{j} \colon j \neq i}\left|a_{i j}\right| \quad(i=1,2, \ldots, n)
$$

であること。つまり、行列のどの対角成分の絶対値も、それと同じ列にある非対角成分の和よりも大きい時。

In [69]:
G = inv(X)*A*X; #G
norm(diag(G) - λ,Inf) # Gの対角成分は固有値λにほぼ一致する

4.24192376735972e-14

#### (2)$A\cdot\bX$の包含$C\in\IC^{n\times n}$を求める。

``方法1``各演算を区間演算に変更した方式(参考：[interval_dot-mul.ipynb](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/interval_dot-mul.html "タイトル"))

In [70]:
using IntervalArithmetic, BenchmarkTools

iA = map(Interval, A);
iX = map(Interval, X);

@btime C1 = $iA * $iX;

  83.545 ms (8 allocations: 312.91 KiB)


In [71]:
@show max(maximum(radius.(real(C1[:]))),maximum(radius.(imag(C1[:]))))

LoadError: UndefVarError: C1 not defined

``方法2``BLASを使う。int_num_linalg.jlの呼び出し。この方が早い。

In [72]:
using BenchmarkTools
include("int_num_linalg.jl"); # int_mulを使用する
# @time C1 = mm_comp_interval_alt(iA,iX);
@time C = int_mul(A,X); # A, X: complex matrices (not interval)

  0.299349 seconds (340.56 k allocations: 16.017 MiB)


In [73]:
max(maximum(radius.(real(C[:]))),maximum(radius.(imag(C[:]))))

9.070522111187529e-14

In [74]:
real(C)

100×100 Array{Interval{Float64},2}:
  [0.141042, 0.141043]     …   [0.0570442, 0.0570443]
 [-0.120578, -0.120577]       [-0.422209, -0.422208]
  [0.316564, 0.316565]        [-0.216498, -0.216497]
  [0.917078, 0.917079]         [0.709749, 0.70975]
  [2.60322, 2.60323]           [0.183854, 0.183855]
  [1.69664, 1.69665]       …  [-0.312201, -0.3122]
  [0.621876, 0.621877]         [0.0681427, 0.0681428]
 [-0.342672, -0.342671]        [0.277068, 0.277069]
 [-0.527738, -0.527737]        [0.963645, 0.963646]
  [0.2627, 0.262701]           [0.0394793, 0.0394794]
 [-0.553732, -0.553731]    …  [-0.293339, -0.293338]
 [-1.00189, -1.00188]         [-0.276278, -0.276277]
 [-1.23231, -1.2323]           [0.491184, 0.491185]
   ⋮                       ⋱  
  [0.461776, 0.461777]         [0.225717, 0.225718]
  [1.30543, 1.30544]           [1.46385, 1.46386]
 [-1.18795, -1.18794]      …   [0.54367, 0.543671]
  [1.01348, 1.01349]          [-0.640185, -0.640184]
  [0.25973, 0.259731]          [0.142198, 0

In [75]:
imag(C)

100×100 Array{Interval{Float64},2}:
  [0.0992197, 0.0992198]   …  [-0.190448, -0.190447]
 [-0.235341, -0.23534]         [0.768363, 0.768364]
  [0.399879, 0.39988]         [-0.354442, -0.354441]
  [0.0386075, 0.0386076]       [0.405916, 0.405917]
  [0.196411, 0.196412]        [-0.62913, -0.629129]
  [0.481859, 0.48186]      …   [0.325322, 0.325323]
 [-0.535315, -0.535314]        [0.258784, 0.258785]
  [0.053509, 0.0535091]        [0.301258, 0.301259]
 [-0.793959, -0.793958]        [0.346126, 0.346127]
  [0.0138863, 0.0138864]      [-0.215598, -0.215597]
 [-0.0638003, -0.0638002]  …   [0.70481, 0.704811]
  [0.720496, 0.720497]        [-0.077436, -0.0774359]
 [-0.0277473, -0.0277472]     [-0.122831, -0.12283]
   ⋮                       ⋱  
  [0.334992, 0.334993]        [-0.028405, -0.0284049]
  [0.117961, 0.117962]         [0.551534, 0.551535]
 [-0.171117, -0.171116]    …  [-0.27936, -0.279359]
 [-0.737791, -0.73779]         [0.3362, 0.336201]
 [-0.281973, -0.281972]       [-0.119983, -0.

#### 3)区間連立1次方程式 $\hat XG=C$ の解集合の包含 $G\supset\hat X^{-1}\cdot C$ を求める。

In [76]:
function verifylss_ib(A,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = A\b;
    n = length(x̄);
    R = inv(A);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - int_mul(R,A);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_ib (generic function with 1 method)

In [77]:
G = similar(C);
for i = 1:n
    G[:,i] = verifylss_ib(X,C[:,i]);
end
real(G)

100×100 Array{Interval{Float64},2}:
 [-10.7661, -10.766]           …  [-1.20376e-12, 1.19351e-12]
  [-1.36823e-12, 1.373e-12]       [-1.18546e-12, 1.21525e-12]
  [-1.72012e-12, 1.72931e-12]     [-1.52936e-12, 1.53534e-12]
  [-1.72284e-12, 1.72715e-12]     [-1.53246e-12, 1.53143e-12]
  [-1.46338e-12, 1.46928e-12]     [-1.3144e-12, 1.283e-12]
  [-1.46186e-12, 1.46901e-12]  …  [-1.30706e-12, 1.28489e-12]
  [-4.15053e-12, 4.1562e-12]      [-3.71743e-12, 3.69997e-12]
  [-4.15167e-12, 4.15768e-12]     [-3.71497e-12, 3.7012e-12]
  [-3.18329e-12, 3.18543e-12]     [-2.78636e-12, 2.81595e-12]
  [-1.30406e-12, 1.30779e-12]     [-1.1552e-12, 1.1579e-12]
  [-1.3068e-12, 1.30355e-12]   …  [-1.15281e-12, 1.16468e-12]
  [-2.2909e-12, 2.28778e-12]      [-2.0163e-12, 2.03774e-12]
  [-2.292e-12, 2.28524e-12]       [-2.01574e-12, 2.0349e-12]
    ⋮                          ⋱  
  [-5.65495e-12, 5.60863e-12]     [-4.91407e-12, 4.91859e-12]
  [-5.65898e-12, 5.60935e-12]     [-4.9147e-12, 4.92321e-12]
  [-2.09

In [78]:
[real(diag(G)), λ]

2-element Array{Array{T,1} where T,1}:
 Interval{Float64}[[-10.7661, -10.766], [-10.7661, -10.766], [-8.9012, -8.90119], [-8.9012, -8.90119], [-7.79643, -7.79642], [-7.79643, -7.79642], [-7.39293, -7.39292], [-7.39293, -7.39292], [-7.32677, -7.32676], [-7.15472, -7.15471]  …  [7.13827, 7.13828], [7.13827, 7.13828], [7.5658, 7.56581], [7.5658, 7.56581], [7.92565, 7.92566], [7.92565, 7.92566], [8.3051, 8.30511], [8.3051, 8.30511], [9.0018, 9.00181], [9.0018, 9.00181]]
 Complex{Float64}[-10.766022723425223 - 0.8122874492930389im, -10.766022723425223 + 0.8122874492930389im, -8.901198365927415 - 2.472701572517997im, -8.901198365927415 + 2.472701572517997im, -7.796420635615081 - 6.605672668300338im, -7.796420635615081 + 6.605672668300338im, -7.39292803449457 - 1.0096385968492632im, -7.39292803449457 + 1.0096385968492632im, -7.326769092147958 + 0.0im, -7.154711828575033 - 5.267615306775259im  …  7.138270608605898 - 6.496864665555203im, 7.138270608605898 + 6.496864665555203im, 7.56580857111344

In [79]:
C=real(C)+imag(C)
C[1,1]

[0.240262, 0.240263]

#### 4)$G\in\IC^{n\times n}$に対してゲルシュゴリンの定理を適用することを考える。


## ゲルシュゴリンの包含定理
行列 $A=(A_{ij})\in \C^{n\times n}$について、$A$の全ての固有値$\lambda_i$は、$\bigcup_{1\le i \le n}  U_i$の内部に存在する。

$$
A = \bigcup_{1\le i \le n}  U_i
$$

ただし、

$$
U_i = \{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\}
$$

である。

円盤領域$U_i$をゲルシュゴリン円板といい、Aが強い優対角性を持つとき、Aの対角成分がAの固有値の良い近似となる。

今回の場合、$G=(g_{ij})$の対角成分$[g_{ii}]$を$c_i$、半径を$\rm{rad(g_{ii})}$とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&c_i\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(g_{ii})\\
\end{array}
$$

となる。

In [98]:
r = randn(n)
for i = 1:n
    mr = mag.(real(G[i,:]))
    mi = mag.(imag(G[i,:]))
    magri = sum(mr + mi)-mag.(real(diag(G)[i]))-mag.(imag(diag(G)[i]))
    ra = radius.(real(diag(G)[i])) + radius.(imag(diag(G)[i]))
    r[i] = magri+ra
end

In [3]:
using IntervalArithmetic
interval(-1,1)-interval(0)

[-1, 1]

In [108]:
c=real(diag(G))+imag(diag(G))

100-element Array{Interval{Float64},1}:
 [-11.5784, -11.5783]
  [-9.95374, -9.95373]
 [-11.3739, -11.3738]
  [-6.4285, -6.42849]
 [-14.4021, -14.402]
  [-1.19075, -1.19074]
  [-8.40257, -8.40256]
  [-6.38329, -6.38328]
  [-7.32677, -7.32676]
 [-12.4224, -12.4223]
  [-1.8871, -1.88709]
 [-10.02, -10.0199]
  [-3.23645, -3.23644]
    ⋮
   [5.97511, 5.97512]
   [7.31101, 7.31102]
   [0.641405, 0.641406]
  [13.6351, 13.6352]
   [3.9882, 3.98821]
  [11.1434, 11.1435]
   [7.37298, 7.37299]
   [8.47832, 8.47833]
   [5.47111, 5.47112]
  [11.139, 11.1391]
   [7.8926, 7.89261]
  [10.111, 10.1111]

In [83]:
using Plots
plot!(index, max_rad, yscale=:log10,
    xlabel       ="Real",                #X軸のラベル
    ylabel       ="Imaginary",   #Y軸のラベル
    xlims        =(0,41),             #X軸の範囲
    title="gershgorin",       #タイトル
    linewidth    =2,                  #線幅
    seriestype = :scatter,            #点プロットに　
    size         =(400,300),          #プロットのサイズ
)

LoadError: UndefVarError: index not defined

In [117]:
function circle(center, radius; color = black, fill::Bool = true, options...)
    # draw
end

circle (generic function with 1 method)

In [118]:
circle(c[1], radius[1]; color = black, fill::Bool = true, option)

LoadError: syntax: invalid keyword argument name "fill::Bool" around In[118]:1

対角優位
http://www.math.ritsumei.ac.jp/~yasutomi/jugyo/Numerical_Analysis/note5.pdf

計算途中がでる
http://www.cas.cmc.osaka-u.ac.jp/~paoon/misc/julia/post/cg/


http://infsup.jp/saito/materials/na19exer.pdf